In [110]:
import pandas as pd
import pickle
import numpy as np
p_processed=pickle.load(open('../prepared-data/p_processed', 'rb'))
n_processed=pickle.load(open('../prepared-data/n_processed', 'rb'))
vocabulary=pickle.load(open('../prepared-data/vocabulary', 'rb'))

#POS subjective vs objective score (higher means more subjective) added 1 to each
tag_scores={'NPS':1-0.82,'WP$':1-0.7,'POS':1-0.45,'NP':1-0.39,'NNS':1-0.32,'IN':1-0.21,'VBN':1-0.21,
   'VBZ':1-0.15,'JJR':1-0.1,'RBS':1-0.03,'TO':1+0.01,'NN':1+0.02,'JJS':1+0.05,'JJ':1+0.1,
   'DT':1+0.11,'VBG':1+0.15,'VBD':1+0.19,'WDT':1+0.21,'WP':1+0.23,'RP':1+0.24,'MD':1+0.3,'VB':1+0.3,
   'RBR':1+0.31,'CC':1+0.33,'EX':1+0.37,'VBP':1+0.51,'WRB':1+0.57,'PP$':1+0.59,'RB':1+0.6,'PP':1+0.78,
   'PDT':1+0.79,'UH':1+0.83}

# Feature Processing
Pre-processed result will have
    words, hashtag, pos tag, negations, word emphasizes

Once we get the word features, we now have to represent the tweet using them. We can have multiple variations of such representation
1. Mean of tfidf
2. Weighted sum of tfidf-s by hashtags, pos_tags, negations, emphasizes
3. 2 with additional parameters
4. Considering multiple words

In [96]:
def feature_processor1(tweetB):
    try:
        return vocabulary.loc[tweetB[0]].mean(axis=0)
    except:
        return [-999,-999]
p_features1=np.array([feature_processor1(tweet) for tweet in p_processed])
print("Half Finished")
n_features1=np.array([feature_processor1(tweet) for tweet in n_processed])

Half Finished


In [111]:
[sum(p_features1[:,0]>p_features1[:,1])/100000,sum(n_features1[:,0]<n_features1[:,1])/100000]

[0.35881999999999997, 0.96758999999999995]

In [112]:
def pos2impact(tags):
    return np.array([tag_scores[tag] if tag in tag_scores.keys() else 1 for tag in tags])
def feature_processor2(tweetB):
    try:
        tweet=vocabulary.loc[tweetB[0]].fillna(0)
        weights=np.array(tweetB[1])+pos2impact(tweetB[2])+tweetB[3]+tweetB[4].astype(int)
        return [(tweet.p*weights).mean(),(tweet.n*weights).mean()]
    except:
        return [-999,-999]
p_features2=np.array([feature_processor2(tweet) for tweet in p_processed])
print("Half Finished")
n_features2=np.array([feature_processor2(tweet) for tweet in n_processed])

Half Finished


In [113]:
[sum(p_features2[:,0]>p_features2[:,1])/100000,sum(n_features2[:,0]<n_features2[:,1])/100000]

[0.36427999999999999, 0.96647000000000005]

In [115]:
def voc_finder(tweet):
    try:
        return vocabulary.loc[tweet[0]].fillna(0)
    except:
        return [-999,-999]
ptweet_vocs=[voc_finder(tweet) for tweet in p_processed]
ntweet_vocs=[voc_finder(tweet) for tweet in n_processed]

In [145]:
not True

False

In [163]:
def cons_finder(tweet):
    return [pos2impact(tweet[2]),70*np.array(tweet[1])+14*tweet[3]+10*tweet[4].astype(int)] #np.array(tweetB[1])+pos2impact(tweetB[2])+tweetB[3]+tweetB[4].astype(int)
p_cons=[cons_finder(tweet) for tweet in p_processed]
n_cons=[cons_finder(tweet) for tweet in n_processed]
print("Started calcuation")
#Parameter estimation
def calc(p,voc,var,cons):
    if(type(voc)!=type([])):
        w=p*var+cons
        return (voc.p*w).mean()>(voc.n*w).mean()
    else:
        return False
def checker(ranges,p_tweet_vocs,n_tweet_vocs):
    for p in ranges:
        pp=sum([calc(p,voc,cons[0],cons[1]) for voc,cons in zip(p_tweet_vocs,p_cons)])
        nn=sum([not calc(p,voc,cons[0],cons[1]) for voc,cons in zip(n_tweet_vocs,n_cons)])
        print("Score:{s},Parameter:{par}".format(s=pp+nn,par=p))
checker(np.arange(0.001,0.01,0.001),ptweet_vocs,ntweet_vocs)

Started calcuation
Score:137148,Parameter:0.001
Score:137030,Parameter:0.002


KeyboardInterrupt: 

A=70
Score:133082,Parameter:0
Score:133165,Parameter:10
Score:133186,Parameter:20
Score:133218,Parameter:30
Score:133246,Parameter:40
Score:133252,Parameter:50
Score:133263,Parameter:60
Score:133292,Parameter:70
Score:133287,Parameter:80
Score:133306,Parameter:90

B=0.001
Score:108009,Parameter:0.0
Score:137148,Parameter:0.001
Score:137030,Parameter:0.002
Score:136865,Parameter:0.01
Score:136797,Parameter:0.02
Score:136786,Parameter:0.03
Score:136605,Parameter:0.4
Score:135372,Parameter:0.8
Score:135354,Parameter:1.2000000000000002
Score:135057,Parameter:1.6

C=14
Score:132708,Parameter:0
Score:133078,Parameter:1
Score:133549,Parameter:2
Score:133907,Parameter:3
Score:134234,Parameter:4
Score:134471,Parameter:5
Score:134673,Parameter:6
Score:134860,Parameter:7
Score:134943,Parameter:8
Score:135035,Parameter:9
Score:135177,Parameter:10
Score:135278,Parameter:11
Score:135354,Parameter:12
Score:135390,Parameter:13
Score:135422,Parameter:14
Score:135275,Parameter:15

D=10
Score:133069,Parameter:0
Score:133082,Parameter:2
Score:133081,Parameter:4
Score:133078,Parameter:6
Score:133082,Parameter:8
Score:133087,Parameter:10
Score:133082,Parameter:12
Score:133086,Parameter:14
Score:133079,Parameter:16
Score:133070,Parameter:18

In [165]:
a,b,c,d=70,0.001,14,10

def feature_processor3(tweetB):
    try:
        tweet=vocabulary.loc[tweetB[0]].fillna(0)
        weights=a*np.array(tweetB[1])+b*pos2impact(tweetB[2])+c*tweetB[3]+d*tweetB[4].astype(int)
        return [(tweet.p*weights).mean(),(tweet.n*weights).mean()]
    except:
        return [-999,-999]

p_features3=np.array([feature_processor3(tweet) for tweet in p_processed])
print("Half Finished")
n_features3=np.array([feature_processor3(tweet) for tweet in n_processed])

Half Finished


In [167]:
[sum(p_features3[:,0]>p_features3[:,1])/100000,sum(n_features3[:,0]<n_features3[:,1])/100000]

[0.56152000000000002, 0.80993000000000004]

So according to our weighting evaluation, the separability has increased around 2 percent

In [ ]:
def tweet2words(tweetB):
    return word_scores
def getwindow(word_scores,i,n):
    return word_scores
def feature_processor4(tweetB,ngram=1):
    word_scores=tweet2words
    total=np.array([0]*ngram)
    for i in range(0,len(word_scores)):
        total+=getwindow(word_scores,i,ngram)
    return total/len(word_scores)